# 決算情報から高ROA銘柄の抽出

最近、Python界隈で野球やサッカーのデータ分析が流行っていて楽しそうなので僕もPandasで遊んでみたくなったという話です。
先日のBPStulyでの池内さんのサッカーの分析とか中川さんの野球分析の話を聞いていて触発されたかんじです。
あとここ1ヶ月くらいは機械学習の勉強していて、この辺の分野はまだまだオモシロイなぁと思う次第です。

## 銘柄抽出の理由

数カ月前に財務会計の本を読んだ以来、貸借対照表を見たりするのが楽しいのでやっています。
モータースポーツが好きなのでそういう数字で遊んでも良かったんですが、あんまり良いデータが見つからなかったので決算情報にしました。
証券会社の提供しているスクリーニングツールには限界があるので自分の欲しい情報が取れない場合が多んです。
もちろん今回やるようなROAでの抽出なんかはできるんですけど、よくある話なので練習に良いかなと。

データは [決算プロ](http://ke.kabupro.jp) というとこから拝借しています。
ライセンスについては調べても分からなかったので、データは再配布せずにあくまでも計算したNotebookだけ公開します。
いくつかデータをピックアップして有価証券報告書と見比べてみてもデータはあっているようだったので、大変ありがたく使わせてもらってます。

## ROAを見たい理由

ROAというは総資産のうち、いくら純利益がありますか？という指標です。
手持ちのお金から会社がどれだけお金を稼いでるかが分かります。
例えば会社が年間20％稼いでくれているとすると、話的には会社の分割である株の価値も年間20％あがるということになります。
もちろん株価は割高、割安がありますし色んな要因で変動するので、ROAが20％の株を買えば年間20％儲かるという話ではないですよ。
ですがまぁオモシロイ話じゃないですか。一体どんな会社さんがでてくるんでしょうか。

In [2]:
import pandas as pd

In [3]:
data = pd.read_excel("./data/20161103f.xls")

In [4]:
data.columns

Index(['証券コード', '企業名', '会計基準', '連結個別', '決算期', '決算期間', '期首', '期末',
       '名寄前勘定科目（売上高欄に掲載）', '売上高', '営業利益', '経常利益', '純利益', '一株当り純利益',
       '希薄化後一株当り純利益', '純資産又は株主資本', '総資産', '一株当り純資産', '営業キャッシュフロー',
       '投資キャッシュフロー', '財務キャッシュフロー', '情報公開又は更新日'],
      dtype='object')

In [5]:
data["企業名"].unique()

array(['極洋', '日本水産', 'マルハ', ..., 'ルネサスイーストン', 'サトー商会', 'ベルーナ'], dtype=object)

## 通年決算情報のみの抽出

ここでデータから、通期決算でかつ連結決算の行だけ取って変数に入れておきます。
Pandasは操作しやすいExcelみたいな感じで便利。

In [6]:
annual_data = data[(data["決算期間"] == "通期") & (data["連結個別"] == "連結")]

In [7]:
annual_data["決算期"].unique()

array(['2016年3月期', '2015年3月期', '2014年3月期', '2016年5月期', '2015年5月期',
       '2014年5月期', '2016年6月期', '2015年6月期', '2014年6月期', '2015年10月期',
       '2016年8月期', '2015年8月期', '2014年8月期', '2015年12月期', '2014年12月期',
       '2013年12月期', '2014年10月期', '2016年2月期', '2015年2月期', '2014年2月期',
       '2015年9月期', '2014年9月期', '2016年4月期', '2015年4月期', '2014年4月期',
       '2016年1月期', '2015年1月期', '2014年1月期', '2013年6月期', '2016年7月期',
       '2015年7月期', '2014年7月期', '2015年11月期', '2014年11月期', '2016年9月期'], dtype=object)

## ROA, ERの算出

純利益 / 総資産 * 100 でROAを計算、
純資産又は株主資本 / 総資産で自己資本比率(ER)を計算しています

In [8]:
annual_data["ROA"] = annual_data["純利益"] / annual_data["総資産"] * 100

/home/hirokiky/dev/pydata_now/env/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [9]:
annual_data["ER"] = annual_data["純資産又は株主資本"] / annual_data["総資産"] * 100

/home/hirokiky/dev/pydata_now/env/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


## 日本水産、日本製粉のROA

ここで日本水産、日本製粉のROAを見てみます。
スーパーでもおなじみのニッスイとニップンです。
何となく思いつきで2つ上げてみました。

In [10]:
annual_data[annual_data["企業名"] == "日本水産"][["企業名", "決算期", "ROA", "ER"]]

,企業名,決算期,ROA,ER
19,日本水産,2016年3月期,2.698151,24.501992
23,日本水産,2015年3月期,2.237569,22.692486
27,日本水産,2014年3月期,0.869700,19.398438


In [11]:
annual_data[annual_data["企業名"] == "日本製粉"][["企業名", "決算期", "ROA", "ER"]]

,企業名,決算期,ROA,ER
3118,日本製粉,2016年3月期,3.316406,54.752964
3122,日本製粉,2015年3月期,2.804854,55.259352
3126,日本製粉,2014年3月期,3.443517,53.630684


## 高ROA銘柄の抽出

ではここで高ROAの銘柄を探してみましょう。
ROAが20％以上、自己資本比率が40％以上を条件にしてPandasで調べてみました。

In [12]:
annual_data[lambda r: ((r["ROA"] >= 20) & (r["ER"] >= 40))][["企業名", "決算期", "ROA", "ER"]]

,企業名,決算期,ROA,ER
437,住石ホールディングス,2015年3月期,22.274747,56.259255
3409,ミクシィ,2016年3月期,36.974291,73.612904
3413,ミクシィ,2015年3月期,31.643917,51.421605
3444,ジェイエイシーリクルートメント,2014年12月期,20.531432,71.328581
3448,ジェイエイシーリクルートメント,2013年12月期,21.077739,74.929329
3454,日本Ｍ＆Ａセンター,2016年3月期,24.030584,83.228241
3458,日本Ｍ＆Ａセンター,2015年3月期,23.582090,80.388060
3462,日本Ｍ＆Ａセンター,2014年3月期,23.925020,78.006725
3584,クルーズ,2014年3月期,32.161163,62.617714
5350,カカクコム,2016年3月期,33.673496,82.541192


## 感想

できました。たしかに証券会社のスクリーニングツールを使って出る結果と一致している気がします。
ただ高ROA銘柄を見てみると新興のサービス系企業が目立ちますし、これだけ見て実際に投資するのは良くなさそうです。

Pandasを使うと自分の手元で手軽にデータをスクリーニングして調べれるので、自分の計算式で銘柄抽出できるのはいいなぁと思います。
まぁこれで銘柄を絞って、そこからIR情報を深く見ていくのが良いんじゃないですかね。
あとは現在の株価とかもジョインして、PBRとか買いの株価はいくらかとか調べれると株の売買に役立つ情報になるんじゃないでしょうか。
